# Simulação de Distribuição de Poisson: Experimentação e Visualização

Este notebook apresenta simulações práticas da distribuição de Poisson, permitindo que você experimente e visualize os conceitos em ação. Utilizaremos o padrão BDD (Behavior Driven Development) para estruturar nossos cenários de forma clara e didática.

## 📚 Recordando: O que é a Distribuição de Poisson?

A distribuição de Poisson modela o número de eventos que ocorrem em um intervalo fixo de tempo ou espaço, quando:
- Os eventos são **independentes** entre si
- A taxa média de ocorrência é **constante** (λ)
- Os eventos são **raros** (baixa probabilidade individual)
- O número de oportunidades é **muito grande**

**Fórmula:** P(X = k) = (e^(-λ) × λ^k) / k!

**Propriedades:** μ = λ e σ² = λ (média e variância são iguais)

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
import pandas as pd
from IPython.display import display, Markdown
import math

# Configuração dos gráficos
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Bibliotecas importadas com sucesso!")
print("Versões utilizadas:")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Pandas: {pd.__version__}")

---
## 📞 Cenário 1: Central de Atendimento

**DADO** que uma central de atendimento recebe em média 4 ligações por minuto  
**E** as ligações chegam de forma independente e aleatória  
**QUANDO** analisamos um período de 1 minuto  
**ENTÃO** queremos determinar as probabilidades de diferentes números de ligações

In [ ]:
# DADO: Parâmetros do cenário
lambda_ligacoes = 4  # média de ligações por minuto
nome_cenario = "Central de Atendimento"

print(f"📞 {nome_cenario}")
print(f"Taxa média de chegada (λ): {lambda_ligacoes} ligações/minuto")
print(f"Média (μ): {lambda_ligacoes}")
print(f"Variância (σ²): {lambda_ligacoes}")
print(f"Desvio padrão (σ): {np.sqrt(lambda_ligacoes):.3f}")

# E: Definindo o espaço amostral (até 15 ligações é suficiente para visualização)
k_max = 15
k_valores = np.arange(0, k_max + 1)
print(f"\nAnalisando possibilidades de 0 a {k_max} ligações por minuto")

In [ ]:
# QUANDO: Calculamos as probabilidades para cada número de ligações
probabilidades = []
detalhes_calculo = []

print("🧮 Calculando probabilidades passo a passo:\n")

for k in k_valores:
    # Cálculo manual usando a fórmula
    e_negativo_lambda = math.exp(-lambda_ligacoes)
    lambda_elevado_k = lambda_ligacoes ** k
    k_fatorial = math.factorial(k)
    prob_manual = (e_negativo_lambda * lambda_elevado_k) / k_fatorial
    
    # Usando scipy para verificação
    prob_scipy = poisson.pmf(k, lambda_ligacoes)
    
    probabilidades.append(prob_manual)
    
    if k <= 8:  # Mostrar detalhes apenas para os primeiros valores
        print(f"P(X = {k}) = (e^(-{lambda_ligacoes}) × {lambda_ligacoes}^{k}) / {k}!")
        print(f"       = ({e_negativo_lambda:.6f} × {lambda_elevado_k} × {k_fatorial})")
        print(f"       = {prob_manual:.6f} ({prob_manual*100:.2f}%)")
        print(f"Verificação (scipy): {prob_scipy:.6f}\n")
    
    detalhes_calculo.append({
        'k': k,
        'λ^k': lambda_elevado_k,
        'k!': k_fatorial,
        'e^(-λ)': e_negativo_lambda,
        'Probabilidade': prob_manual,
        'Percentual': prob_manual * 100
    })

# Verificação: soma deve ser próxima de 1 (aproximada pois truncamos em k_max)
soma_prob = sum(probabilidades)
print(f"✅ Verificação: Soma das probabilidades (0 a {k_max}) = {soma_prob:.6f}")
print(f"   Probabilidade de k > {k_max}: {1 - soma_prob:.6f}")

In [ ]:
# ENTÃO: Visualizando e analisando os resultados
df_resultado = pd.DataFrame(detalhes_calculo[:10])  # Mostrar apenas os 10 primeiros

print("📋 Tabela de Resultados (primeiras 10 possibilidades):")
display(df_resultado.round(6))

# Criando visualizações
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Distribuição de probabilidades (stem plot)
markerline, stemlines, baseline = ax1.stem(k_valores, probabilidades, basefmt=" ")
markerline.set_markerfacecolor('red')
markerline.set_markersize(8)
stemlines.set_color('blue')
stemlines.set_linewidth(2)

ax1.set_title(f'{nome_cenario}\nDistribuição de Poisson (λ = {lambda_ligacoes})', fontsize=12, fontweight='bold')
ax1.set_xlabel('Número de ligações (k)')
ax1.set_ylabel('Probabilidade P(X = k)')
ax1.grid(True, alpha=0.3)

# Destacando o valor mais provável
max_prob_idx = np.argmax(probabilidades)
ax1.plot(max_prob_idx, probabilidades[max_prob_idx], 'go', markersize=12, label=f'Moda = {max_prob_idx}')
ax1.axvline(lambda_ligacoes, color='red', linestyle='--', alpha=0.7, label=f'λ = {lambda_ligacoes}')
ax1.legend()

# Gráfico 2: Comparação com distribuição normal
x_continuo = np.linspace(0, k_max, 1000)
y_normal = (1/np.sqrt(2*np.pi*lambda_ligacoes)) * np.exp(-0.5*((x_continuo - lambda_ligacoes)**2)/lambda_ligacoes)

ax2.bar(k_valores, probabilidades, alpha=0.7, color='lightblue', label='Poisson')
ax2.plot(x_continuo, y_normal, 'r-', linewidth=2, label=f'Normal(μ={lambda_ligacoes}, σ²={lambda_ligacoes})')
ax2.set_title('Comparação: Poisson vs Normal', fontsize=12, fontweight='bold')
ax2.set_xlabel('Número de ligações (k)')
ax2.set_ylabel('Densidade/Probabilidade')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Função de distribuição acumulada
cdf_valores = poisson.cdf(k_valores, lambda_ligacoes)
ax3.step(k_valores, cdf_valores, where='post', linewidth=3, color='green')
ax3.fill_between(k_valores, cdf_valores, step='post', alpha=0.3, color='green')
ax3.set_title('Função de Distribuição Acumulada', fontsize=12, fontweight='bold')
ax3.set_xlabel('k')
ax3.set_ylabel('P(X ≤ k)')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 1.05)

# Linhas de interesse
ax3.axhline(0.5, color='red', linestyle='--', alpha=0.7, label='50%')
ax3.axhline(0.95, color='orange', linestyle='--', alpha=0.7, label='95%')
ax3.legend()

# Gráfico 4: Análise de intervalos de interesse
intervalos = {
    'P(X = λ)': probabilidades[lambda_ligacoes],
    'P(X ≤ λ)': sum(probabilidades[:lambda_ligacoes+1]),
    'P(X > λ)': sum(probabilidades[lambda_ligacoes+1:]),
    'P(X ≥ 2λ)': sum(probabilidades[2*lambda_ligacoes:]) if 2*lambda_ligacoes < len(probabilidades) else 0,
    'P(X = 0)': probabilidades[0],
    'P(1 ≤ X ≤ 6)': sum(probabilidades[1:7])
}

labels = list(intervalos.keys())
valores = list(intervalos.values())
cores = plt.cm.Set3(np.linspace(0, 1, len(labels)))

bars = ax4.barh(labels, valores, color=cores)
ax4.set_title('Probabilidades de Interesse', fontsize=12, fontweight='bold')
ax4.set_xlabel('Probabilidade')

# Adicionando valores nas barras
for bar, valor in zip(bars, valores):
    ax4.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{valor:.3f}\n({valor*100:.1f}%)', ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\n📊 Análise dos Resultados:")
print(f"• Número mais provável de ligações: {max_prob_idx}")
print(f"• Probabilidade de receber exatamente {lambda_ligacoes} ligações: {probabilidades[lambda_ligacoes]*100:.2f}%")
print(f"• Probabilidade de receber 0 ligações: {probabilidades[0]*100:.2f}%")
print(f"• Probabilidade de receber mais que a média (>{lambda_ligacoes}): {sum(probabilidades[lambda_ligacoes+1:])*100:.2f}%")
print(f"• Probabilidade de sobrecarga (≥8 ligações): {sum(probabilidades[8:])*100:.2f}%")

---
## 🏭 Cenário 2: Defeitos em Produção

**DADO** que uma fábrica de tecidos tem em média 2 defeitos por metro de tecido  
**E** os defeitos ocorrem de forma aleatória e independente  
**QUANDO** analisamos diferentes comprimentos de tecido  
**ENTÃO** queremos prever o número de defeitos esperados

In [ ]:
# DADO: Parâmetros do cenário de controle de qualidade
lambda_defeitos_metro = 2  # média de defeitos por metro
comprimentos = [0.5, 1, 2, 3, 5]  # diferentes comprimentos a analisar
nome_cenario_2 = "Controle de Qualidade - Tecidos"

print(f"🏭 {nome_cenario_2}")
print(f"Taxa média de defeitos: {lambda_defeitos_metro} defeitos/metro")
print(f"Comprimentos analisados: {comprimentos} metros")

# E: Simulação para diferentes comprimentos
print(f"\n🎲 Simulando defeitos para diferentes comprimentos...")
np.random.seed(42)  # Para reprodutibilidade

resultados_comprimentos = {}
for comp in comprimentos:
    lambda_atual = lambda_defeitos_metro * comp  # λ ajustado para o comprimento
    simulacao = np.random.poisson(lambda_atual, 1000)
    resultados_comprimentos[comp] = {
        'lambda': lambda_atual,
        'simulacao': simulacao,
        'media_teorica': lambda_atual,
        'media_simulacao': np.mean(simulacao),
        'desvio_teorico': np.sqrt(lambda_atual),
        'desvio_simulacao': np.std(simulacao)
    }

print("Simulações realizadas para todos os comprimentos.")

In [ ]:
# QUANDO: Analisamos as distribuições para cada comprimento
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

estatisticas_comprimentos = []

for i, comp in enumerate(comprimentos):
    dados = resultados_comprimentos[comp]
    lambda_comp = dados['lambda']
    simulacao = dados['simulacao']
    
    # Calculando distribuição teórica
    k_max_comp = max(15, int(lambda_comp + 4*np.sqrt(lambda_comp)))
    k_vals = np.arange(0, k_max_comp + 1)
    prob_teoricas = poisson.pmf(k_vals, lambda_comp)
    
    # Plotando
    ax = axes[i]
    
    # Histograma da simulação
    ax.hist(simulacao, bins=np.arange(-0.5, max(simulacao)+1.5, 1), 
            density=True, alpha=0.6, color='lightblue', 
            edgecolor='blue', label=f'Simulação (1000 amostras)')
    
    # Distribuição teórica
    valid_k = k_vals[k_vals <= max(simulacao)+2]
    valid_prob = prob_teoricas[k_vals <= max(simulacao)+2]
    ax.plot(valid_k, valid_prob, 'ro-', linewidth=2, markersize=5, label='Teórico')
    
    ax.axvline(lambda_comp, color='red', linestyle='--', alpha=0.8, 
               label=f'λ = {lambda_comp}')
    ax.axvline(dados['media_simulacao'], color='green', linestyle='--', alpha=0.8, 
               label=f'Média sim. = {dados["media_simulacao"]:.2f}')
    
    ax.set_title(f'Comprimento: {comp}m (λ = {lambda_comp})', fontweight='bold')
    ax.set_xlabel('Número de defeitos')
    ax.set_ylabel('Densidade/Probabilidade')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Coletando estatísticas
    estatisticas_comprimentos.append({
        'Comprimento (m)': comp,
        'λ': lambda_comp,
        'Média Teórica': dados['media_teorica'],
        'Média Simulação': dados['media_simulacao'],
        'Desvio Teórico': dados['desvio_teorico'],
        'Desvio Simulação': dados['desvio_simulacao'],
        'P(X=0)': poisson.pmf(0, lambda_comp),
        'P(X≥5)': 1 - poisson.cdf(4, lambda_comp)
    })

# Último gráfico: comparação das médias
ax = axes[5]
medias_teoricas = [dados['media_teorica'] for dados in resultados_comprimentos.values()]
medias_simulacao = [dados['media_simulacao'] for dados in resultados_comprimentos.values()]

x_comp = np.arange(len(comprimentos))
width = 0.35

bars1 = ax.bar(x_comp - width/2, medias_teoricas, width, 
               label='Média Teórica', color='lightcoral', alpha=0.8)
bars2 = ax.bar(x_comp + width/2, medias_simulacao, width, 
               label='Média Simulação', color='lightblue', alpha=0.8)

ax.set_title('Comparação de Médias por Comprimento', fontweight='bold')
ax.set_xlabel('Comprimento (metros)')
ax.set_ylabel('Média de defeitos')
ax.set_xticks(x_comp)
ax.set_xticklabels(comprimentos)
ax.legend()
ax.grid(True, alpha=0.3)

# Linha de tendência
ax.plot(x_comp, [lambda_defeitos_metro * c for c in comprimentos], 
        'g--', linewidth=2, label='Tendência linear')

plt.tight_layout()
plt.show()

# Tabela comparativa
df_comprimentos = pd.DataFrame(estatisticas_comprimentos)
print("📊 Análise Comparativa por Comprimento:")
display(df_comprimentos.round(4))

---
## 🚗 Cenário 3: Acidentes de Trânsito

**DADO** que uma cidade registra em média 3 acidentes por dia  
**E** queremos analisar diferentes períodos de tempo  
**QUANDO** consideramos semanas e meses  
**ENTÃO** podemos planejar recursos de emergência

In [ ]:
# DADO: Parâmetros do cenário de acidentes
lambda_acidentes_dia = 3  # média de acidentes por dia
periodos = {
    '1 dia': 1,
    '1 semana': 7,
    '1 mês': 30,
    '1 trimestre': 90,
    '1 ano': 365
}

nome_cenario_3 = "Acidentes de Trânsito"

print(f"🚗 {nome_cenario_3}")
print(f"Taxa média: {lambda_acidentes_dia} acidentes/dia")
print(f"Períodos analisados: {list(periodos.keys())}")

# Função para análise de risco
def analisar_periodo_acidentes(periodo_nome, dias, lambda_dia=lambda_acidentes_dia):
    lambda_periodo = lambda_dia * dias
    
    # Calculando probabilidades de interesse
    prob_zero = poisson.pmf(0, lambda_periodo)
    prob_ate_media = poisson.cdf(lambda_periodo, lambda_periodo)
    prob_acima_media = 1 - prob_ate_media
    prob_dobro = 1 - poisson.cdf(2*lambda_periodo, lambda_periodo) if 2*lambda_periodo < 1000 else 0
    
    # Percentis para planejamento
    percentil_95 = poisson.ppf(0.95, lambda_periodo)
    percentil_99 = poisson.ppf(0.99, lambda_periodo)
    
    return {
        'periodo': periodo_nome,
        'dias': dias,
        'lambda': lambda_periodo,
        'media': lambda_periodo,
        'desvio_padrao': np.sqrt(lambda_periodo),
        'prob_zero': prob_zero,
        'prob_ate_media': prob_ate_media,
        'prob_acima_media': prob_acima_media,
        'prob_dobro': prob_dobro,
        'percentil_95': percentil_95,
        'percentil_99': percentil_99
    }

# Analisando todos os períodos
analises_periodos = []
for periodo_nome, dias in periodos.items():
    analise = analisar_periodo_acidentes(periodo_nome, dias)
    analises_periodos.append(analise)
    
    print(f"\n📅 {periodo_nome.upper()}:")
    print(f"   λ = {analise['lambda']:.1f} acidentes esperados")
    print(f"   σ = {analise['desvio_padrao']:.2f}")
    print(f"   P(X = 0) = {analise['prob_zero']:.6f}")
    print(f"   95º percentil: {analise['percentil_95']:.0f} acidentes")
    print(f"   99º percentil: {analise['percentil_99']:.0f} acidentes")

In [ ]:
# ENTÃO: Visualização para planejamento de recursos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Crescimento da média e desvio por período
periodos_nomes = [a['periodo'] for a in analises_periodos]
medias = [a['media'] for a in analises_periodos]
desvios = [a['desvio_padrao'] for a in analises_periodos]

x_pos = np.arange(len(periodos_nomes))
bars1 = ax1.bar(x_pos, medias, alpha=0.7, color='lightcoral', label='Média (μ)')
bars2 = ax1.bar(x_pos, desvios, alpha=0.7, color='lightblue', label='Desvio Padrão (σ)')

ax1.set_title('Crescimento da Média e Desvio por Período', fontweight='bold')
ax1.set_xlabel('Período')
ax1.set_ylabel('Número de acidentes')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(periodos_nomes, rotation=45)
ax1.legend()
ax1.set_yscale('log')  # Escala logarítmica devido à grande variação
ax1.grid(True, alpha=0.3)

# Gráfico 2: Percentis para planejamento
percentis_95 = [a['percentil_95'] for a in analises_periodos]
percentis_99 = [a['percentil_99'] for a in analises_periodos]

ax2.plot(x_pos, medias, 'o-', linewidth=2, markersize=8, label='Média', color='blue')
ax2.plot(x_pos, percentis_95, 's-', linewidth=2, markersize=8, label='95º Percentil', color='orange')
ax2.plot(x_pos, percentis_99, '^-', linewidth=2, markersize=8, label='99º Percentil', color='red')

ax2.set_title('Percentis para Planejamento de Recursos', fontweight='bold')
ax2.set_xlabel('Período')
ax2.set_ylabel('Número de acidentes')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(periodos_nomes, rotation=45)
ax2.legend()
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Gráfico 3: Probabilidades de risco
prob_zeros = [a['prob_zero'] for a in analises_periodos[:3]]  # Apenas primeiros 3 (probabilidades não negligíveis)
prob_acima_medias = [a['prob_acima_media'] for a in analises_periodos]

x_risk = np.arange(3)
ax3.bar(x_risk, prob_zeros, alpha=0.8, color='green', label='P(X = 0) - Sem acidentes')
ax3.set_title('Probabilidade de Períodos sem Acidentes', fontweight='bold')
ax3.set_xlabel('Período')
ax3.set_ylabel('Probabilidade')
ax3.set_xticks(x_risk)
ax3.set_xticklabels(periodos_nomes[:3])
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

# Adicionando valores
for bar, prob in zip(ax3.patches, prob_zeros):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 2, 
             f'{prob:.2e}', ha='center', va='bottom', fontsize=10)

# Gráfico 4: Distribuição para 1 semana (exemplo detalhado)
lambda_semana = 7 * lambda_acidentes_dia
k_semana = np.arange(0, 25)
prob_semana = poisson.pmf(k_semana, lambda_semana)

bars = ax4.bar(k_semana, prob_semana, alpha=0.8, color='purple')
ax4.axvline(lambda_semana, color='red', linestyle='--', linewidth=2, label=f'λ = {lambda_semana}')
ax4.axvline(percentis_95[1], color='orange', linestyle='--', linewidth=2, label=f'95º percentil = {percentis_95[1]:.0f}')

ax4.set_title('Distribuição Detalhada - 1 Semana', fontweight='bold')
ax4.set_xlabel('Número de acidentes')
ax4.set_ylabel('Probabilidade')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Destacar áreas de risco
risco_alto = k_semana >= percentis_95[1]
for i, bar in enumerate(bars):
    if risco_alto[i]:
        bar.set_color('red')
        bar.set_alpha(1.0)

plt.tight_layout()
plt.show()

# Tabela de planejamento
df_planejamento = pd.DataFrame(analises_periodos)
colunas_interesse = ['periodo', 'lambda', 'desvio_padrao', 'percentil_95', 'percentil_99', 'prob_zero']
print("\n📋 Tabela para Planejamento de Recursos:")
display(df_planejamento[colunas_interesse].round(6))

print("\n🚨 Interpretação para Gestores:")
print("• 95º percentil: recursos necessários para cobrir 95% dos cenários")
print("• 99º percentil: recursos para emergências extremas")
print("• P(X = 0): probabilidade de período sem acidentes (muito baixa para períodos longos)")
print("• A preparação deve considerar os percentis, não apenas a média")

---
## 🧪 Função Interativa - Experimento Personalizado

**DADO** que queremos experimentar com diferentes valores de λ  
**QUANDO** modificamos os parâmetros  
**ENTÃO** podemos observar como a distribuição se comporta

In [ ]:
def experimento_poisson_interativo(lam, titulo="Experimento Personalizado", k_max=None):
    """
    Função para criar uma análise completa da distribuição de Poisson
    com parâmetros personalizados.
    """
    print(f"🔬 {titulo}")
    print(f"Parâmetro λ = {lam}")
    print("=" * 50)
    
    # Definindo range apropriado
    if k_max is None:
        k_max = max(15, int(lam + 4*np.sqrt(lam)))
    
    k_valores = np.arange(0, k_max + 1)
    probabilidades = poisson.pmf(k_valores, lam)
    
    # Propriedades estatísticas
    media = lam
    variancia = lam
    desvio_padrao = np.sqrt(lam)
    moda = int(np.floor(lam))
    
    # Simulação
    np.random.seed(42)
    simulacao = np.random.poisson(lam, 10000)
    
    # Criando visualizações
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
    
    # Gráfico 1: Distribuição principal
    markerline, stemlines, baseline = ax1.stem(k_valores, probabilidades, basefmt=" ")
    markerline.set_markerfacecolor('red')
    markerline.set_markersize(8)
    stemlines.set_color('blue')
    stemlines.set_linewidth(2)
    
    ax1.axvline(media, color='red', linestyle='--', linewidth=2, label=f'λ = {media}')
    ax1.axvline(moda, color='orange', linestyle='--', linewidth=2, label=f'Moda ≈ {moda}')
    ax1.set_title(f'{titulo}\nDistribuição de Poisson(λ = {lam})', fontweight='bold')
    ax1.set_xlabel('k (número de eventos)')
    ax1.set_ylabel('P(X = k)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Destacando o valor mais provável
    max_prob_idx = np.argmax(probabilidades)
    ax1.plot(max_prob_idx, probabilidades[max_prob_idx], 'go', markersize=12, 
             label=f'Máximo em k={max_prob_idx}')
    
    # Gráfico 2: Simulação vs Teórico
    ax2.hist(simulacao, bins=np.arange(-0.5, max(min(simulacao.max(), k_max), 20)+1.5, 1), 
             density=True, alpha=0.6, color='lightblue', 
             edgecolor='blue', label='Simulação (10,000)')
    
    # Sobrepor distribuição teórica
    k_plot = k_valores[k_valores <= min(simulacao.max() + 5, k_max)]
    prob_plot = probabilidades[k_valores <= min(simulacao.max() + 5, k_max)]
    ax2.plot(k_plot, prob_plot, 'ro-', linewidth=2, markersize=5, label='Teórico')
    
    ax2.set_title('Validação por Simulação', fontweight='bold')
    ax2.set_xlabel('k')
    ax2.set_ylabel('Densidade')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Gráfico 3: Função de distribuição acumulada
    cdf = poisson.cdf(k_valores, lam)
    ax3.step(k_valores, cdf, where='post', linewidth=3, color='purple')
    ax3.fill_between(k_valores, cdf, step='post', alpha=0.3, color='purple')
    
    # Linhas de percentis importantes
    percentis = [0.25, 0.5, 0.75, 0.95]
    cores_perc = ['green', 'orange', 'red', 'black']
    
    for p, cor in zip(percentis, cores_perc):
        ax3.axhline(p, color=cor, linestyle='--', alpha=0.7, label=f'{int(p*100)}%')
    
    ax3.set_title('Função de Distribuição Acumulada', fontweight='bold')
    ax3.set_xlabel('k')
    ax3.set_ylabel('P(X ≤ k)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim(0, 1.05)
    
    # Gráfico 4: Comparação com aproximação normal (se λ ≥ 10)
    if lam >= 10:
        from scipy.stats import norm
        x_cont = np.linspace(max(0, lam - 4*desvio_padrao), lam + 4*desvio_padrao, 1000)
        y_normal = norm.pdf(x_cont, lam, desvio_padrao)
        
        ax4.bar(k_valores, probabilidades, alpha=0.6, color='lightblue', 
                width=0.8, label='Poisson')
        ax4.plot(x_cont, y_normal, 'r-', linewidth=2, 
                 label=f'Normal(μ={lam}, σ={desvio_padrao:.2f})')
        ax4.set_title('Aproximação Normal (λ ≥ 10)', fontweight='bold')
        ax4.set_xlabel('k')
        ax4.set_ylabel('Densidade/Probabilidade')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    else:
        # Para λ pequeno, mostrar probabilidades específicas
        probs_especiais = {
            f'P(X = 0)': poisson.pmf(0, lam),
            f'P(X = 1)': poisson.pmf(1, lam),
            f'P(X ≤ {int(lam)})': poisson.cdf(int(lam), lam),
            f'P(X > {int(lam)})': 1 - poisson.cdf(int(lam), lam),
            f'P(X ≥ {int(2*lam)})': 1 - poisson.cdf(int(2*lam)-1, lam) if int(2*lam) < k_max else 0
        }
        
        labels = list(probs_especiais.keys())
        valores = list(probs_especiais.values())
        cores = plt.cm.Set3(np.linspace(0, 1, len(labels)))
        
        bars = ax4.barh(labels, valores, color=cores, alpha=0.8)
        ax4.set_title('Probabilidades Específicas', fontweight='bold')
        ax4.set_xlabel('Probabilidade')
        
        for bar, valor in zip(bars, valores):
            ax4.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
                     f'{valor:.4f}', ha='left', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Relatório textual
    print(f"\n📊 Relatório de Análise:")
    print(f"• λ (taxa média): {lam}")
    print(f"• Média (μ): {media}")
    print(f"• Variância (σ²): {variancia}")
    print(f"• Desvio Padrão (σ): {desvio_padrao:.3f}")
    print(f"• Moda (aproximada): {moda}")
    print(f"• Valor mais provável: k = {max_prob_idx} com P = {probabilidades[max_prob_idx]:.4f}")
    
    # Percentis importantes
    percentil_50 = poisson.ppf(0.5, lam)
    percentil_95 = poisson.ppf(0.95, lam)
    percentil_99 = poisson.ppf(0.99, lam)
    
    print(f"\n🎯 Percentis Importantes:")
    print(f"• 50º percentil (mediana): {percentil_50:.0f}")
    print(f"• 95º percentil: {percentil_95:.0f}")
    print(f"• 99º percentil: {percentil_99:.0f}")
    
    # Validação da simulação
    media_simulacao = np.mean(simulacao)
    variancia_simulacao = np.var(simulacao)
    
    print(f"\n✅ Validação (Lei dos Grandes Números):")
    print(f"• Média teórica: {media:.4f}")
    print(f"• Média simulada: {media_simulacao:.4f}")
    print(f"• Variância teórica: {variancia:.4f}")
    print(f"• Variância simulada: {variancia_simulacao:.4f}")
    
    return {
        'lambda': lam,
        'probabilidades': probabilidades,
        'k_valores': k_valores,
        'simulacao': simulacao,
        'percentis': {'50': percentil_50, '95': percentil_95, '99': percentil_99}
    }

print("🎮 Função interativa criada! Use-a com diferentes valores de λ.")

In [ ]:
# EXEMPLO 1: Valor pequeno de λ (eventos raros)
resultado1 = experimento_poisson_interativo(lam=1.5, titulo="Eventos Raros (λ = 1.5)")

In [ ]:
# EXEMPLO 2: Valor médio de λ
resultado2 = experimento_poisson_interativo(lam=8, titulo="Valor Médio (λ = 8)")

In [ ]:
# EXEMPLO 3: Valor grande de λ (aproximação normal)
resultado3 = experimento_poisson_interativo(lam=25, titulo="Aproximação Normal (λ = 25)")

---
## 🎯 Exercícios Práticos para Experimentação

Use a função `experimento_poisson_interativo()` para explorar os seguintes cenários:

In [ ]:
print("🎮 EXERCÍCIOS PRÁTICOS:")
print("=" * 60)
print("\n1. 📧 Emails por hora:")
print("   Você recebe em média 12 emails por hora.")
print("   Qual a distribuição esperada?")
print("   Execute: experimento_poisson_interativo(12, 'Emails por Hora')")

print("\n2. 🌟 Estrelas cadentes:")
print("   Durante uma chuva de meteoros, vemos 0.5 estrelas por minuto.")
print("   Em 10 minutos, quantas esperamos ver?")
print("   Execute: experimento_poisson_interativo(5, 'Estrelas em 10min')")

print("\n3. 🚨 Emergências hospitalares:")
print("   Um hospital recebe 18 emergências por dia.")
print("   Como planejar os recursos?")
print("   Execute: experimento_poisson_interativo(18, 'Emergências/Dia')")

print("\n4. 🐛 Bugs no software:")
print("   Um sistema tem em média 0.3 bugs por 1000 linhas de código.")
print("   Em 10.000 linhas, quantos bugs esperamos?")
print("   Execute: experimento_poisson_interativo(3, 'Bugs em 10k linhas')")

print("\n5. ⚡ Quedas de energia:")
print("   Uma região tem 2.2 quedas de energia por mês.")
print("   Qual a probabilidade de diferentes cenários?")
print("   Execute: experimento_poisson_interativo(2.2, 'Quedas/Mês')")

print("\n6. 🔬 Experimento Livre:")
print("   Escolha seu próprio valor de λ e observe os padrões!")
print("   Execute: experimento_poisson_interativo(SEU_LAMBDA, 'Seu Título')")

print("\n" + "="*60)
print("💡 DICAS para exploração:")
print("• λ < 1: eventos muito raros, distribuição muito assimétrica")
print("• 1 ≤ λ ≤ 10: casos típicos da distribuição de Poisson")
print("• λ > 10: distribuição se aproxima da normal")
print("• λ >> 30: aproximação normal é excelente")
print("• Compare cenários com mesmo λ mas diferentes contextos")

In [ ]:
# Espaço para seus experimentos
# Descomente e modifique as linhas abaixo para executar os exercícios:

# experimento_poisson_interativo(12, 'Emails por Hora')
# experimento_poisson_interativo(5, 'Estrelas em 10min')
# experimento_poisson_interativo(18, 'Emergências/Dia')
# experimento_poisson_interativo(3, 'Bugs em 10k linhas')
# experimento_poisson_interativo(2.2, 'Quedas/Mês')
# experimento_poisson_interativo(SEU_LAMBDA, 'Seu Título')

print("🔬 Remova os comentários (#) das linhas acima para executar os experimentos!")
print("✏️ Substitua 'SEU_LAMBDA' pelo valor que desejar testar.")
print("\n🎯 Sugestões de λ para testar: 0.5, 1, 3, 7, 15, 30, 50")

---
## 📝 Resumo e Conclusões

### ✅ O que aprendemos:

1. **Padrão BDD aplicado**: Estruturamos cada cenário com "Dado-Quando-Então" para maior clareza

2. **Cenários práticos explorados**:
   - Central de atendimento (telecomunicações)
   - Controle de qualidade (manufatura) 
   - Acidentes de trânsito (segurança pública)
   - Diversos experimentos personalizáveis

3. **Características fundamentais**:
   - μ = λ (média igual ao parâmetro)
   - σ² = λ (variância igual à média)
   - Aproximação normal para λ grande (≥ 10)
   - Útil para modelar eventos raros

4. **Aplicações para planejamento**:
   - Uso de percentis para dimensionar recursos
   - Análise de risco baseada em probabilidades
   - Previsão para diferentes períodos de tempo

### 🔍 Principais insights:
- **Para λ pequeno (< 5)**: distribuição muito assimétrica, concentrada em valores baixos
- **Para λ médio (5-15)**: distribuição característica de Poisson, moderadamente assimétrica
- **Para λ grande (> 20)**: aproximação normal é excelente, distribuição quase simétrica

### 🎯 Próximos passos:
- Explore a conexão entre Binomial e Poisson (quando n→∞ e p→0, mas np=λ)
- Aplique em dados reais da sua área de interesse
- Compare com outras distribuições para eventos raros

### 📚 Para saber mais:
- Processo de Poisson
- Distribuição Exponencial (tempos entre eventos Poisson)
- Aplicações em teoria de filas
